In [ ]:
import numpy as np
import neal
from pyqubo import Array, solve_qubo, Constraint
from sklearn.metrics import accuracy_score
from itertools import product

class best_SVM:
    def __init__(self, data, labels, B=2, K=2, Xi=1, gamma=0.9, C=1):
        self.B = B
        self.K = K
        self.N = data.shape[0]
        self.Xi = Xi
        self.gamma = float(gamma)
        self.C = C
        self.data = data
        self.labels = labels
        self.alpha = Array.create('alpha', shape=self.K * self.N, vartype='BINARY')

    def rbf_kernel(self, x, y):
        return np.exp(-self.gamma * np.dot(np.subtract(x, y), np.subtract(x, y)))

    def construct_qubo(self):
        alpha = self.alpha
        x = self.data
        t = self.labels
        energy = sum(alpha[self.K * n + k] * alpha[self.K * m + j] * t[n] * t[m] * self.rbf_kernel(x[n], x[m]) * self.B ** (k + j)
                     for n, m in product(range(self.N), repeat=2) for k, j in product(range(self.K), repeat=2))
        const_1 = sum(alpha[self.K * n + k] * self.B ** k for n in range(self.N) for k in range(self.K))
        const_2 = sum(alpha[self.K * n + k] * t[n] * self.B ** k for n in range(self.N) for k in range(self.K)) ** 2
        model = 0.5 * energy - const_1 + self.Xi * const_2
        model = Constraint(model, label='SVM')
        return model.compile().to_qubo()

    def train(self):
        qubo, offset = self.construct_qubo()
        sampleset = neal.SimulatedAnnealingSampler().sample_qubo(qubo, num_reads=10)
        best_sample = sampleset.first.sample
        self.alpha_values = np.array([best_sample[f'alpha[{i}]'] for i in range(self.K * self.N)])
        return self.alpha_values.reshape((self.N, self.K))

    def predict(self, X):
        scores = np.array([sum(self.alpha_values[n] * self.labels[n] * self.rbf_kernel(X[i], self.data[n]) * self.B ** k
                             for n in range(self.N) for k in range(self.K))
                           for i in range(X.shape[0])])
        return np.sign(scores)

    def evaluate(self, X, y):
        predictions = self.predict(X)
        return accuracy_score(y, predictions)

# Define SVM parameters
# params = {
#     "data": data,
#     "label": t,
#     "B": 2,
#     "K": 2,
#     "Xi": 1,
#     "gamma": gamma,
#     "C": 1,
#     "kernel": "rbf",
#     "optimizer": "SA"
# }

model = best_SVM(X_train, y_train, gamma)
model.train()
print("Accuracy on training set:", model.evaluate(X_test, y_test))

In [ ]:
    def PyquboToNparray(self, qubo_dict):
        # qubo generate by pyqubo is tuple type data, this func for changing the qubo from dict to np.array
        # qubometasolver need a (spins_num,spins_num) size np.array type value
        # pyqubo generate a dict type qubo and only has the triangle of qubo like this
        # 2.9, 1.4, 2.6
        # null, 1.7, 5.2
        # null, null, 3.7
        # so dict length is spins*spins/2

        #print("Pyqubo before sort",qubo_dict)

        qubo = np.zeros((self.K * self.N, self.K * self.N))
        for i in range (self.K * self.N):
            for j in range (i, self.K * self.N):
                qubo[i][j] = qubo_dict.get(('alpha[%s]' % i, 'alpha[%s]' % j))
                if math.isnan(qubo[i][j]):
                    qubo[i][j] = qubo_dict.get(('alpha[%s]' % j, 'alpha[%s]' % i))

        return qubo